In [1]:
import numpy as np
from sklearn.manifold import TSNE
import torch
from transformers import BertConfig, BertModel, BertTokenizer
from transformers import BertForMaskedLM, BertTokenizerFast, pipeline
from pprint import pprint
import numpy as np
from bert_serving.client import BertClient
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

/home/customer/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
config = BertConfig.from_json_file("config.json")
model = BertForMaskedLM.from_pretrained('/home/customer/hyr/matbert-uncased（copy-version）')
tokenizer = BertTokenizerFast.from_pretrained('/home/customer/hyr/matbert-uncased（copy-version）', do_lower_case=False)
# unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [ ]:
# 输入单词 并将其转换为token
input_text = "perovskite is used in [MASK]"
input_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors='pt')

# 使用模型预测mask位置的token的隐藏状态
with torch.no_grad():
    outputs = model(input_ids)

# 提取 "perovskite" 对应的词向量
mask_index = (input_ids == tokenizer.mask_token_id).nonzero()[0, 1]  # 获取 "MASK" token 的索引
word_logits = outputs.logits[0, mask_index]

print("词向量：", word_logits)


In [26]:
# 输入单词 并将其转换为token
input_text = "perovskite is used in [MASK]"
input_ids = tokenizer.encode(input_text, add_special_tokens=False, return_tensors='pt')

# 使用模型预测mask位置的token的隐藏状态
with torch.no_grad():
    outputs = model(input_ids)

# 提取 "perovskite" 对应的词向量
mask_index = (input_ids == tokenizer.mask_token_id).nonzero()[0, 1]  # 获取 "MASK" token 的索引
word_logits = outputs.logits[0, mask_index]

print("词向量：", word_logits)

词向量： tensor([-9.9807, -6.4048, -6.3191,  ..., -6.8940, -5.5510, -4.5033])


In [27]:
# 获取BERT模型的词汇表
vocab = tokenizer.get_vocab()
vocab_list = list(vocab.keys())

In [42]:
# 选择一部分词汇进行可视化，或者使用全部词汇也可以
num_words_to_visualize = 10000
vocab_list_subset = vocab_list[:num_words_to_visualize]


# 如果你想要可视化全部词向量↓
# vocab_list_subset = vocab_list

In [43]:
# 获取词汇的词向量
embeddings = []
for word in vocab_list_subset:
    token_id = tokenizer(word, return_tensors="pt")["input_ids"]
    with torch.no_grad():
        output = model(input_ids=token_id)
        embeddings.append(output[0].mean(dim=1).numpy())

embeddings_np = np.concatenate(embeddings, axis=0)



In [ ]:
# from sklearn.cluster import MiniBatchKMeans
# 使用MiniBatchKMeans进行聚类
# num_clusters = 10
# kmeans = MiniBatchKMeans(n_clusters=num_clusters)
# cluster_labels = kmeans.fit_predict(embeddings_np)


In [ ]:
from umap import UMAP

# 使用UMAP进行降维
reducer = UMAP()
embeddings_umap = reducer.fit_transform(embeddings_np)



In [ ]:
# 选取要高亮的词汇
highlight_words = ['material']
# 绘制可视化图
plt.figure(figsize=(10, 8))
for i in range(num_clusters):
    plt.scatter(embeddings_umap[cluster_labels == i, 0], embeddings_umap[cluster_labels == i, 1], label=f'Cluster {i}')

# 高亮显示指定的词汇
highlight_indices = [vocab_list_subset.index(word) for word in highlight_words]
highlight_embeddings = embeddings_umap[highlight_indices]
plt.scatter(highlight_embeddings[:, 0], highlight_embeddings[:, 1], color='red', marker='o', label='Highlighted Words')
for i, word in zip(highlight_indices, highlight_words):
    plt.text(embeddings_umap[i, 0], embeddings_umap[i, 1], word, fontsize=10)

plt.legend()
plt.title('UMAP Visualization of BERT Word Embeddings')
plt.show()

In [ ]:
highlight_words = ['material']
plt.figure(figsize=(10, 8))

# 绘制聚类点
for i in range(num_clusters):
    plt.scatter(embeddings_umap[cluster_labels == i, 0], embeddings_umap[cluster_labels == i, 1], color='gray', label=f'Cluster {i}')

# 高亮显示指定的词汇
highlight_embeddings = embeddings_umap[highlight_indices]
plt.scatter(highlight_embeddings[:, 0], highlight_embeddings[:, 1], color='blue', marker='o', label='Highlighted Words')
for i, word in zip(highlight_indices, highlight_words):
    plt.text(embeddings_umap[i, 0], embeddings_umap[i, 1], word, fontsize=10)

plt.legend()
plt.title('UMAP Visualization of BERT Word Embeddings')
plt.show()


In [ ]:
# 分界线----------------------以下不要kmean了-------------

In [ ]:
# 获取BERT模型的词汇表
vocab = tokenizer.get_vocab()
vocab_list = list(vocab.keys())

In [ ]:
# 选择一部分词汇进行可视化，或者使用全部词汇也可以
num_words_to_visualize = 10000
vocab_list_subset = vocab_list[:num_words_to_visualize]


# 如果你想要可视化全部词向量↓
# vocab_list_subset = vocab_list

In [ ]:
# 获取词汇的词向量
embeddings = []
for word in vocab_list_subset:
    token_id = tokenizer(word, return_tensors="pt")["input_ids"]
    with torch.no_grad():
        output = model(input_ids=token_id)
        embeddings.append(output[0].mean(dim=1).numpy())

embeddings_np = np.concatenate(embeddings, axis=0)

In [ ]:
from umap import UMAP

# 使用UMAP进行降维
reducer = UMAP()
embeddings_umap = reducer.fit_transform(embeddings_np)


In [ ]:
import matplotlib.pyplot as plt

# Words to highlight in blue
blue_highlight_words = ['perovskite', 'word1', 'word2']

# Words to highlight in green
green_highlight_words = ['word3', 'word4', 'word5']

# Words to highlight in yellow
yellow_highlight_words = ['word6', 'word7', 'word8']

# Function to determine color based on word
def get_color(word):
    if word in blue_highlight_words:
        return 'blue'
    elif word in green_highlight_words:
        return 'green'
    elif word in yellow_highlight_words:
        return 'yellow'
    else:
        return 'gray'

# Plot the embeddings
plt.figure(figsize=(10, 8))
for i, word in enumerate(vocab_list_subset):
    color = get_color(word)
    plt.scatter(embeddings_umap[i, 0], embeddings_umap[i, 1], c=color)
    if color != 'gray':
        plt.text(embeddings_umap[i, 0], embeddings_umap[i, 1], word, fontsize=9, color=color)
plt.title('UMAP Visualization of Word Embeddings')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.show()


In [ ]:
# ---------------------------新分界线---下面要画渐变色图

In [ ]:
import numpy as np
import torch
from umap import UMAP
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# Function to compute cosine similarity between vectors
def cosine_similarity_vec(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# Calculate cosine similarity between "perovskite" and all other words
perovskite_embedding = None
perovskite_index = None
for i, word in enumerate(vocab_list_subset):
    if word == "perovskite":
        perovskite_embedding = embeddings_np[i]
        perovskite_index = i
        break


In [ ]:
cosine_similarities = []
for i, embedding in enumerate(embeddings_np):
    if i != perovskite_index:
        similarity = cosine_similarity_vec(perovskite_embedding, embedding)
        cosine_similarities.append((i, similarity))


In [ ]:
# Sort cosine similarities by value and get the top 1000
cosine_similarities.sort(key=lambda x: x[1], reverse=True)
top_similarities = cosine_similarities[:1000]


In [ ]:
# Extract words and similarities
top_words = [vocab_list_subset[i[0]] for i in top_similarities]
top_values = [i[1] for i in top0_similarities]


In [ ]:
# Plot the embeddings with cosine similarity as color
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_umap[:, 0], embeddings_umap[:, 1], c=top_1000_values, cmap='Purples', alpha=0.6)

# Highlight 'HTL' and 'ETL' with star markers
htl_index = vocab_list_subset.index('HTL')
etl_index = vocab_list_subset.index('ETL')
plt.scatter(embeddings_umap[htl_index, 0], embeddings_umap[htl_index, 1], c='red', marker='*', label='HTL')
plt.scatter(embeddings_umap[etl_index, 0], embeddings_umap[etl_index, 1], c='blue', marker='*', label='ETL')

plt.colorbar(label='Cosine Similarity')
plt.title('UMAP Visualization of Word Embeddings with Cosine Similarity to "perovskite"')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.legend()
plt.savefig('colorful.png')
plt.show()